# Units mystery in the Priestley-Taylor equation 

Author: Stan Schymanski, stan.schymanski@datascience.ch

In this worksheet, we explore the famous Priestley-Taylor equation (Priestley and Taylor, 1972) using the Environmental Science for SageMath (essm) package, downloadable from https://github.com/environmentalscience/essm.

In [1]:
from IPython.display import display, Math

from essm.equations import Equation
from essm.variables import Variable
from essm.variables.utils import generate_metadata_table
from essm.variables.units import markdown, joule, kelvin, kilogram, meter, pascal, second

Below, we use essm `Variable` class to define relevant variables used in the paper by Priestley and Taylor. Since units were not provided in the paper, we assume standard SI units for each variable. Since $q_s$ was referred to as specific humidity at saturation by Priestley and Taylor (1972), we assume units of kg m$^{-3}$. Consequently, since $s=\partial q_s/\partial T$, the units of $s$ are assumed to be kg m$^{-3}$ K$^{-1}$.

In [2]:
# Import standard variables from physics module
from essm.variables.physics.thermodynamics import lambda_E, c_pa

# Define custom variables
class E_w(Variable):
    """Wet surface evaporation
    (E in Priestley and Taylor, 1972))
    """
    unit = kilogram/meter^2/second
    latex_name = 'E'
    
class H_w(Variable):
    """Wet surface sensible heat flux (positive outwards)"""
    unit = joule/second/meter^2
    latex_name = 'H'
      
class q_a(Variable):
    """Specific humidity"""
    unit = kilogram/meter^3
    
class q_s(Variable):
    """Specific humidity at saturation"""
    unit = kilogram/meter^3
    
class T(Variable):
    """Temperature"""
    unit = kelvin 
    
class s_PT(Variable):
    """Priestley-Taylor $\partial q_s/\partial T$"""
    unit = kilogram/meter^3/kelvin
    latex_name = 's'

# Re-define `lambda_E` as `L_E` to have the same latex representation as in the paper
class L_E(lambda_E.definition):
    """Latent heat"""   
    latex_name = 'L'
    default = lambda_E.definition.default
    unit = lambda_E.definition.unit
    
# Define `gamma_PT = c_pa/L_E` as in the paper
class gamma_PT(Variable):
    """Priestley-Taylor $\gamma$"""
    expr = c_pa/L_E
    latex_name = r'\gamma'

In the above, we defined the variable $\gamma$ exactly in the same way as in Priestley and Taylor (1972), i.e. $\gamma = c_p/L$, and called the variable `gamma_PT`.

In [3]:
table(list(generate_metadata_table([E_w, H_w, L_E, c_pa, s_PT, T, gamma_PT])))

Symbol,Name,Description,Default value,Units
,gamma_PT,Priestley-Taylor,-,K
,c_pa,Specific heat of dry air.,1010.0,J K kg
,E_w,"Wet surface evaporation (E in Priestley and Taylor, 1972))",-,kg m s
,H_w,Wet surface sensible heat flux (positive outwards),-,J m s
,L_E,Latent heat,2450000.0,J kg
,s_PT,Priestley-Taylor,-,kg K m
,T,Temperature,-,K


Eq. 3 was given in the paper as:
$$ \frac{LE}{H} = \frac{s}{\gamma}$$

Using the variables defined above, we can write it as:

In [4]:
eq3 = L_E*E_w/H_w == s_PT/gamma_PT
display(Math(latex(eq3)))

<IPython.core.display.Math object>

Now, we will try to define the above expression as a physical equation, using our `Equation` class:

In [5]:
try: 
    class eq_3(Equation):
        """Equation 3 in Priestley and Taylor (1972)"""
        expr = L_E*E_w/H_w == s_PT/gamma_PT
except Exception, error:
    print error

Invalid expression units: 1 == kilogram/meter^3


Since $EL/H$ is non-dimensional, the right hand side of Eq. 3 must be non-dimensional, too. Clearly, the units of Eq. 3 do not match if we use our assumptions about the units of $L$, $s$ and $c_{pa}$. Either the equation is missing a division by a density term (units of kg m$^{-3}$) on the right hand side, or one of our assumptions about the units involved was different to what the authors had in mind. 

Note that the definition of $\gamma \equiv c_{pa}/L$, which was entered above when defining `gamma_PT`, is stored in a dictionary, `Variable.__expressions__` and is also directly associated with the variable through `gamma_PT.definition.expr`. The units of each variable can be accessed in a similar way, through `Variable.__units__` or `gamma_PT.definition.unit`.

In [6]:
(gamma_PT == gamma_PT.definition.expr).show()
print gamma_PT.definition.unit

gamma_PT == c_pa/L_E

1/kelvin


We can substitute the stored definition of `gamma_PT` into `eq3` and verify that the unit problem does not go away, by also substituting the units for the variables.:

In [7]:
eq_3a = eq3.subs(Variable.__expressions__)
eq_3a.show()
eq_3a.subs(Variable.__units__)

E_w*L_E/H_w == L_E*s_PT/c_pa

1 == kilogram/meter^3

**There is no obvious way of interpreting the variables in Equation 3 in a way that the units on both sides of the equation would balance.**